In [11]:
# Step 1: Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

In [13]:
# Read the data from the Excel file into a DataFrame
data_file = './AlzheimerData.xlsx'
OriginalMatrix = pd.read_excel(data_file, header=None)  # No header specified

In [15]:
# Step 2: Print OriginalMatrix
print("OriginalMatrix:")
print(OriginalMatrix)

OriginalMatrix:
         0      1      2   3    4   5     6       7    8   9     10    11  \
0        Y      X1     X2  X3   X4  X5    X6      X7   X8  X9   X10   X11   
1    517.06  9.233   1.13   0    7  16     4   0.067    4   0    12     2   
2    504.06  9.001  1.129   0    8  16     4   0.068    4   0    12     1   
3    490.03  9.246  1.128   0    8  16     3   0.054    4   0    12     1   
4   1443.08  6.414  1.146   0  !!!  20    42   0.187  sds   0     6   qwe   
5    517.06  9.233   1.13   0    7  16     4   0.067    4   0    12     2   
6    514.57    8.3  1.127   0    7  23     5   0.076    5   0    18     4   
7    518.09  8.781  1.129   0    8  16     5     !!!    4   0    12     1   
8    531.09  9.002   1.13   0    7  16     4   0.063    4   0  #@@^     2   
9    516.59  8.072  1.129   0    7  22     5   0.074    4   0    18     4   
10   505.05  9.183  1.132   0    8  16     4   0.069    4   0    12     2   
11   519.08   8.95  1.133   0    9  16     4     $$$    4   

In [46]:
# Remove columns that have all zeros or at least 5 junks
# # Define a function to identify "junk" values in the data
is_junk = OriginalMatrix.apply(lambda x: pd.to_numeric(x, errors='coerce').isna().sum() >= 5, axis=0) # Convert non-numeric values to NaN and check if there are at least 5 NaNs
all_zeros = (OriginalMatrix == 0).all(axis=0)
# Create Matrix1 by selecting columns from OriginalMatrix
# Exclude columns that are identified as junk or contain all zeros
Matrix1 = OriginalMatrix.loc[:, ~(is_junk | all_zeros)] # Select columns that are neither junk nor all zeros

In [19]:
# Step 4: Print Matrix1
print("\nMatrix1 after removing columns with all zeros or at least 5 junks:")
print(Matrix1)


Matrix1 after removing columns with all zeros or at least 5 junks:
         0      1      2   3    4   5     6    8   9     10    11    13    14  \
0        Y      X1     X2  X3   X4  X5    X6   X8  X9   X10   X11   X13   X14   
1    517.06  9.233   1.13   0    7  16     4    4   0    12     2  39.3   3.6   
2    504.06  9.001  1.129   0    8  16     4    4   0    12     1  39.3   1.8   
3    490.03  9.246  1.128   0    8  16     3    4   0    12     1  39.6   1.9   
4   1443.08  6.414  1.146   0  !!!  20    42  sds   0     6   qwe  wqew  ^^^^   
5    517.06  9.233   1.13   0    7  16     4    4   0    12     2  39.3   3.6   
6    514.57    8.3  1.127   0    7  23     5    5   0    18     4  46.8   6.5   
7    518.09  8.781  1.129   0    8  16     5    4   0    12     1    39   1.7   
8    531.09  9.002   1.13   0    7  16     4    4   0  #@@^     2    39   3.4   
9    516.59  8.072  1.129   0    7  22     5    4   0    18     4  45.3   6.3   
10   505.05  9.183  1.132   0    8  16   

In [21]:
# Step 5: Remove rows that have only zeros or at least 5 junks
is_row_junk = Matrix1.apply(lambda x: pd.to_numeric(x, errors='coerce').isna().sum(), axis=1) >= 5 # Check if the count of NaNs is 5 or more
only_zeros = (Matrix1 == 0).all(axis=1) # Check if all values in each row are zero
Matrix2 = Matrix1.loc[~(is_row_junk | only_zeros)]  # Select rows that are neither junk nor all zeros

In [23]:
# Step 6: Print Matrix2
print("\nMatrix2 after removing rows with only zeros or at least 5 junks:")
print(Matrix2)


Matrix2 after removing rows with only zeros or at least 5 junks:
        0      1      2  3   4   5  6  8  9     10 11    13   14   15
1   517.06  9.233   1.13  0   7  16  4  4  0    12  2  39.3  3.6  7.1
2   504.06  9.001  1.129  0   8  16  4  4  0    12  1  39.3  1.8  7.1
3   490.03  9.246  1.128  0   8  16  3  4  0    12  1  39.6  1.9  7.5
5   517.06  9.233   1.13  0   7  16  4  4  0    12  2  39.3  3.6  7.1
6   514.57    8.3  1.127  0   7  23  5  5  0    18  4  46.8  6.5  4.8
7   518.09  8.781  1.129  0   8  16  5  4  0    12  1    39  1.7  6.8
8   531.09  9.002   1.13  0   7  16  4  4  0  #@@^  2    39  3.4  6.8
9   516.59  8.072  1.129  0   7  22  5  4  0    18  4  45.3  6.3  4.7
10  505.05  9.183  1.132  0   8  16  4  4  0    12  2  38.2  3.6  7.3
11  519.08   8.95  1.133  0   9  16  4  4  0    12  2  37.9  3.4  6.9
12  490.55  8.176  1.131  0   8  22  6  4  0    18  4    45  6.7    5
13  551.66  8.113  1.161  0   8   9  4  3  0     6  3  36.8  4.4  2.9
14  503.07  8.826  1.125

In [25]:
# Step 7: Replace junks in Matrix2 with the average value of that column
Matrix3 = Matrix2.apply(pd.to_numeric, errors='coerce')  # Convert non-numeric to NaN
Matrix3 = Matrix3.fillna(Matrix3.mean())  # Replace NaN (junks) with column mean

In [27]:
# Step 8: Print Matrix3
print("\nMatrix3 after replacing junks in columns with the column mean:")
print(Matrix3)


Matrix3 after replacing junks in columns with the column mean:
        0      1      2   3   4   5   6   8   9          10  11    13   14  \
1   517.06  9.233  1.130   0   7  16   4   4   0  12.000000   2  39.3  3.6   
2   504.06  9.001  1.129   0   8  16   4   4   0  12.000000   1  39.3  1.8   
3   490.03  9.246  1.128   0   8  16   3   4   0  12.000000   1  39.6  1.9   
5   517.06  9.233  1.130   0   7  16   4   4   0  12.000000   2  39.3  3.6   
6   514.57  8.300  1.127   0   7  23   5   5   0  18.000000   4  46.8  6.5   
7   518.09  8.781  1.129   0   8  16   5   4   0  12.000000   1  39.0  1.7   
8   531.09  9.002  1.130   0   7  16   4   4   0  13.333333   2  39.0  3.4   
9   516.59  8.072  1.129   0   7  22   5   4   0  18.000000   4  45.3  6.3   
10  505.05  9.183  1.132   0   8  16   4   4   0  12.000000   2  38.2  3.6   
11  519.08  8.950  1.133   0   9  16   4   4   0  12.000000   2  37.9  3.4   
12  490.55  8.176  1.131   0   8  22   6   4   0  18.000000   4  45.0  6.7   


In [29]:
# Step 9: Replace junks in Matrix3 with the average value of that row
Matrix4 = Matrix3.T.apply(lambda x: x.fillna(x.mean())).T  # Replace NaN (junks) with row mean

In [31]:
# Step 10: Print Matrix4
print("\nMatrix4 after replacing junks in rows with the row mean:")
print(Matrix4)


Matrix4 after replacing junks in rows with the row mean:
        0      1      2    3     4     5    6    8    9          10   11  \
1   517.06  9.233  1.130  0.0   7.0  16.0  4.0  4.0  0.0  12.000000  2.0   
2   504.06  9.001  1.129  0.0   8.0  16.0  4.0  4.0  0.0  12.000000  1.0   
3   490.03  9.246  1.128  0.0   8.0  16.0  3.0  4.0  0.0  12.000000  1.0   
5   517.06  9.233  1.130  0.0   7.0  16.0  4.0  4.0  0.0  12.000000  2.0   
6   514.57  8.300  1.127  0.0   7.0  23.0  5.0  5.0  0.0  18.000000  4.0   
7   518.09  8.781  1.129  0.0   8.0  16.0  5.0  4.0  0.0  12.000000  1.0   
8   531.09  9.002  1.130  0.0   7.0  16.0  4.0  4.0  0.0  13.333333  2.0   
9   516.59  8.072  1.129  0.0   7.0  22.0  5.0  4.0  0.0  18.000000  4.0   
10  505.05  9.183  1.132  0.0   8.0  16.0  4.0  4.0  0.0  12.000000  2.0   
11  519.08  8.950  1.133  0.0   9.0  16.0  4.0  4.0  0.0  12.000000  2.0   
12  490.55  8.176  1.131  0.0   8.0  22.0  6.0  4.0  0.0  18.000000  4.0   
13  551.66  8.113  1.161  0.0 

In [48]:
# Step 11: Rescale each column of Matrix4 using StandardScaler from sklearn
scaler = StandardScaler() # Initialize the StandardScaler object
Matrix5 = pd.DataFrame(scaler.fit_transform(Matrix4), columns=Matrix4.columns) # Apply scaling to Matrix4 and Preserve the original column names in the new DataFrame

# Print Matrix5 after rescaling
print("\nMatrix5 after rescaling each column:")
print(Matrix5)


Matrix5 after rescaling each column:
          0         1         2    3         4         5         6         8   \
0   0.264591  1.167039 -0.089109  0.0 -0.053916 -0.272622 -0.689441  0.308607   
1  -0.454549  0.727411 -0.209668  0.0  0.566122 -0.272622 -0.689441  0.308607   
2  -1.230667  1.191673 -0.330228  0.0  0.566122 -0.272622 -1.622214  0.308607   
3   0.264591  1.167039 -0.089109  0.0 -0.053916 -0.272622 -0.689441  0.308607   
4   0.126848 -0.600945 -0.450787  0.0 -0.053916  1.556215  0.243332  2.083095   
5   0.321569  0.310524 -0.209668  0.0  0.566122 -0.272622  0.243332  0.308607   
6   1.040708  0.729306 -0.089109  0.0 -0.053916 -0.272622 -0.689441  0.308607   
7   0.238591 -1.032992 -0.209668  0.0 -0.053916  1.294953  0.243332  0.308607   
8  -0.399784  1.072291  0.152010  0.0  0.566122 -0.272622 -0.689441  0.308607   
9   0.376334  0.630769  0.272569  0.0  1.186161 -0.272622 -0.689441  0.308607   
10 -1.201901 -0.835918  0.031450  0.0  0.566122  1.294953  1.176105  0.